### Regridding all the sea ice concentration data to EASE

- The SIC data was downloaded from NSIDc for all the days between april15 and october15
- The below notebook converts it from polar stereographic to EASE grid, puts it into an xarray dataset and saves it as a netcdf file to be used in ice_energy_abs.py script

In [8]:
from scipy.interpolate import griddata
import pyproj as proj
import numpy as np
import xarray as xr
import glob
import struct

args = proj.Proj(proj="laea", lat_0=90, lon_0=0, datum="WGS84", units="m")

crs_wgs = proj.Proj(init='epsg:4326')  # assuming you're using WGS84 geographic


# Robbie's regridding function for converting the sea ice concentration data in polar stereographic
# to the EASE grid used by APP-x

def regrid(data_in,
           lon_in,
           lat_in,
           lon_out,
           lat_out,
           method='nearest'):

    xout, yout = proj.transform(crs_wgs, args, np.array(lon_out),np.array(lat_out))

    xin, yin = proj.transform(crs_wgs, args, np.array(lon_in),np.array(lat_in))

    output = griddata((xin.ravel(),yin.ravel()),
                    np.array(data_in).ravel(),
                    (xout,yout),
                    method=method)
    
    return(output)




# PS dims
dimX = 448
dimY = 304

# Fetch PS lon and lat data
PS_lat = (np.fromfile('/Users/farrerowsleybrown/Desktop/project/code/SIC/psn25lats_v3.dat',
                      dtype='<i4').reshape(dimX,dimY))/100000
PS_lon = (np.fromfile('/Users/farrerowsleybrown/Desktop/project/code/SIC/psn25lons_v3.dat',
                      dtype='<i4').reshape(dimX,dimY))/100000

# Fetch EASE lon and lat data
EASE_data = xr.open_dataset('/Users/farrerowsleybrown/Desktop/project/code/grids/EASE_grid.nc')
EASE_lon, EASE_lat = EASE_data.lon.data, EASE_data.lat.data


# Function that reads binary SIC data, converts it to EASE grid and puts it in xarray dataset ready to
# be exported as netcdf for analysis
def sic_toEASE(year):
    get_coords = xr.open_dataset('/Users/farrerowsleybrown/Desktop/project/code/data/app-x/2011/4/'+
                                 'Polar-APP-X_v02r00_Nhem_0400_d20110329_c20190625.nc')
    get_coords.close()
    sic_xr = xr.Dataset() # creates xarray dataset with correct coordinates
    sic_xr = sic_xr.assign_coords(get_coords.drop('time').coords)
    sic_xr = sic_xr.assign_coords({'dayofyear':np.array(range(105,289))})
    
    icefiles = sorted(glob.glob('/Users/farrerowsleybrown/Desktop/project/code/data/nsidc_sic/'
                                +str(year)+'/*.bin'))
    
    SIC = np.array([])
    for file in icefiles:
        icefile = open(file, 'rb') # read binary file
        contents = icefile.read()
        s="%dB" % (int(dimX*dimY),)
        z=struct.unpack_from(s, contents, offset = 300) # takes out header
        new = np.array(z).reshape((dimX,dimY)) # reshapes into PS grid dimensions
        new = new/250 # converts SIC to number between 0 and 1
        new[new>1]=np.nan # takes out land/coastline mask
        
        data_EASE = regrid(new, PS_lon, PS_lat, EASE_lon, EASE_lat) # regrid to EASE
        
        SIC = np.append(SIC, data_EASE)
    
    SIC = np.reshape(SIC, (184,361,361)) # 184 days
    sic_xr['sic'] = (('dayofyear', 'columns', 'rows'), SIC)
    
    return sic_xr

In [ ]:
# loop over all the years to convert
for year in range(1982, 2018):
    ds = sic_toEASE(year)
    ds.to_netcdf(path='/Users/farrerowsleybrown/Desktop/project/code/data/my_data/sic_EASE_nc/sic_EASE_'+
                str(year)+'.nc')
    ds.close()
    print('done ', year)